In [1]:
# import the necessary packages
from transform import four_point_transform
from skimage.filters import threshold_local
import numpy as np
import argparse
import cv2
import imutils
import matplotlib.pyplot as plt

### Edge Detection

In [ ]:
# load the image and compute the ratio of the old height
# to the new height, clone it, and resize it
image = cv2.imread("page4.jpg")

In [ ]:
#plt.figure(figsize=(10, 20))
#imgplot = plt.imshow(image)

In [ ]:
ratio = image.shape[0] / 1000
orig = image.copy()
image = imutils.resize(image, height = 1000)

In [ ]:
# convert the image to grayscale, blur it, and find edges
# in the image
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(gray, 75, 200)

In [ ]:
#plt.figure(figsize=(10, 20))
#imgplot = plt.imshow(edged)

In [ ]:
# show the original image and the edge detected image
cv2.imshow("Image", image)
cv2.imshow("Edged", edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Find edges of pages (contours)

In [ ]:
# find the contours in the edged image, keeping only the
# largest ones, and initialize the screen contour
cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]

In [ ]:
# loop over contours
for c in cnts:
    # approximate the contour
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02*peri, True)
    
    # if our approximated contour has 4 points, then we can assume we have foound the page
    if len(approx) == 4:
        screenCnt = approx
        break

In [ ]:
# show the contour (outline) of the piece of paper
cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
cv2.imshow("Outline", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Threshold

In [ ]:
# convert the warped image to grayscale, then threshold it
# to give it that 'black and white' paper effect
warped = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
T = threshold_local(warped, 11, offset = 10, method = "gaussian")
warped = (warped > T).astype("uint8") * 255

In [ ]:
# show the original and scanned images
print("STEP 3: Apply perspective transform")
cv2.imshow("Original", imutils.resize(orig, height = 650))
cv2.imshow("Scanned", imutils.resize(warped, height = 650))
cv2.waitKey(0)

### PyTesseract

In [2]:
# import the necessary packages
from PIL import Image
import pytesseract
import argparse
import cv2
import os

In [7]:
# load the example image and convert it to grayscale
image = cv2.imread("page2.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# make a check to see if median blurring should be done to remove
# noise
#gray = cv2.medianBlur(gray, 3)
gray = cv2.GaussianBlur(gray, (3, 3), 0)

# check to see if we should apply thresholding to preprocess the
# image
gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
 
# write the grayscale image to disk as a temporary file so we can
# apply OCR to it
filename = "test.png"
cv2.imwrite(filename, gray)

True

In [8]:
# load the image as a PIL/Pillow image, apply OCR, and then delete
# the temporary file
text = pytesseract.image_to_string(Image.open(filename))
os.remove(filename)
print(text)

38 THE LEAN STARTUP

the investors who allocate
trepreneurial teams. It isc
and small—that depend
You can’t take learning t
You cannot give it to ¢

Precious Money, time
old comfort to the o

On entrepreneurial INNovatio
0 the bank; you cant spend j
ustomers and cannot return it to limited
partners. Is it any wonder that learning has a bad name in entre.
preneurial and managerial circles?

Yet if the fundamental goal of entre

in organization building under conditions of extreme uncer.
tainty, its most vital function is learning. We must learn the
truth about which elements of our strategy are working to te-
alize our vision and which are Just crazy. We must learn what
customers really want, not what they say they want or what we
think they should want. We must discover whether we are on
path that will lead to growing a sustainable business. |
In the Lean Startup model, we are rehabilitating learning
with a concept I call validated learning. Validated learning is not
after-the-fact rat

In [10]:
import cv2 
import pytesseract

In [13]:
# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

#skew correction
def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

#template matching
def match_template(image, template):
    return cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)

In [21]:
img = cv2.imread('page6.jpg')

# Adding custom options
custom_config = r'--oem 3 --psm 6'

gray = get_grayscale(img)
deskewed = deskew(gray)
thresh = thresholding(deskewed)

In [15]:
pytesseract.image_to_string(img, config=custom_config)

"rice is one of the fastest-growing companies of all time. Its\n\nname comes from “group coupons,” an ingenious idea that\nhas spawned an entire industry of social commerce imitators.\nHowever, it didn’t start out successful. When customers took\nGroupon up on its first deal, a whopping twenty people bought\ntwo-for-one pizza in a restaurant on the first floor of the com-\npany’ Chicago offices—hardly a world-changing event.\n\nIn fact, Groupon wasn‘ originally meant to be about com-\nmerce at all. The founder, Andrew Mason, intended his com-\npany to become a “collective activism platform” called The\nPoint. Its goal was to bring people together to solve problems\nthey couldn't solve on their own, such as fund-raising for a\ncause or boycotting a certain retailer. The Point's early re-\nsults were disappointing, however, and at the end of 2008 the\nfounders decided to try something new. Although they still\nhad grand ambitions, they were determined to keep the new\nproduct simple. The

In [22]:
pytesseract.image_to_string(thresh, config=custom_config)

'finer is one of the fastest-growing companies of all time. Its\n\nname comes from “group coupons,” an ingenious idea that\nhas spawned an entire industry of social commerce imitators.\nHowever, it didn’t start out successful. When customers took\nGroupon up on its first deal, a whopping twenty people bought\ntwo-for-one pizza in a restaurant on the first floor of the com-\npany’s Chicago offices—hardly a world-changing event.\n\nIn fact, Groupon wasn’ originally meant to be about com-\nmerce at all. The founder, Andrew Mason, intended his com-\npany to become a “collective activism platform” called The\nPoint. Its goal was to bring people together to solve problems\nthey couldn’t solve on their own, such as fund-raising for a\ncause or boycotting a certain retailer. The Point’s early re-\nsults were disappointing, however, and at the end of 2008 the\nfounders decided to try something new. Although they still\nhad grand ambitions, they were determined to keep the new\nproduct simple. T